In [1]:
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [2]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [12]:
#Read and transform the train data
df_train = pd.read_parquet('/content/drive/MyDrive/MLOPs Zoomcamp/Data/yellow_tripdata_2022-01.parquet')
df_train.shape

(2463931, 19)

In [13]:
df_train['duration'] = df_train.tpep_dropoff_datetime - df_train.tpep_pickup_datetime
df_train.duration = df_train.duration.apply(lambda td: td.total_seconds() / 60)

df_train = df_train[(df_train.duration >= 1) & (df_train.duration <= 60)]

categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

df_train[categorical] = df_train[categorical].astype(str)
df_train.shape

(2421440, 20)

In [14]:
df_train.head(5)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,N,142,236,1,14.5,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0,17.816667
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,N,236,42,1,8.0,0.5,0.5,4.00,0.0,0.3,13.30,0.0,0.0,8.400000
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,7.5,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0,8.966667
3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,8.0,0.5,0.5,0.00,0.0,0.3,11.80,2.5,0.0,10.033333
4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,1.0,N,68,163,1,23.5,0.5,0.5,3.00,0.0,0.3,30.30,2.5,0.0,37.533333


In [15]:
#Read and transform the validation data
df_valid = pd.read_parquet('/content/drive/MyDrive/MLOPs Zoomcamp/Data/yellow_tripdata_2022-02.parquet')

df_valid['duration'] = df_valid.tpep_dropoff_datetime - df_valid.tpep_pickup_datetime
df_valid.duration = df_valid.duration.apply(lambda td: td.total_seconds() / 60)

df_valid = df_valid[(df_valid.duration >= 1) & (df_valid.duration <= 60)]

categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

df_valid[categorical] = df_valid[categorical].astype(str)

In [16]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_valid['PU_DO'] = df_valid['PULocationID'] + '_' + df_valid['DOLocationID']

In [17]:
categorical = ['PU_DO']

2. Standard deviation of the trips duration in January?

In [18]:
np.std(df_train['duration'])

8.999279782053803

In [19]:
#Perform One hot encoding
dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_valid[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [20]:
X_train.shape

(2421440, 20667)

In [22]:
X_train

<2421440x20667 sparse matrix of type '<class 'numpy.float64'>'
	with 4842880 stored elements in Compressed Sparse Row format>

In [24]:
target = 'duration'
y_train = df_train[target].values
y_val = df_valid[target].values

In [30]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred_val = lr.predict(X_val)

In [31]:
y_pred_train = lr.predict(X_train)

In [33]:
from math import sqrt
rmse_train = sqrt(mean_squared_error(y_train, y_pred_train, squared=False))
rmse_valid = sqrt(mean_squared_error(y_val, y_pred_val, squared=False))
print(rmse_train)
print(rmse_valid)

2.1902664062725394
2.3516446983227
